In [168]:
import os
import csv
import json
import pandas as pd
import datetime
from google.cloud import storage
from supabase import create_client

In [54]:
# connect to gcloud bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'survai-data-connect.json'
storage_client = storage.Client()
bucket_name = 'survai-dataset'
bucket = storage_client.bucket(bucket_name)

# Connect to database
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
email = os.environ.get("EMAIL")
password = os.environ.get("PASSWORD")
client = create_client(url, key)
user = client.auth.sign_in(email=email, password=password)

In [56]:
table = 'patriot_front_data'

In [182]:
# access temp_videodata_storage folder
blobs = storage_client.list_blobs(bucket_name, prefix='datasets/test_datasets/sparverius_pf/results_08052022/temp_videodata_storage/')
file_list = []

for blob in blobs:

    file = blob.name.split('/')[5]

    # filter for desired files
    if 'od.json' in file or 'stats.json' in file:
        i = file.count('_') # count instances of '_' in file name

        if i > 1:
            file_name = "_".join(file.split("_", i)[:i]) # split on last '_'
            if file_name not in file_list:
                file_list.append(file_name)
                client.table(table).insert({'video': f'{file_name}.mp4'}).execute()

        else:
            file_name = file.split('_')[0]
            if file_name not in file_list:
                file_list.append(file_name)
                client.table(table).insert({'video': f'{file_name}.mp4'}).execute()

            
    if 'od.json' in file:
        od = json.loads(blob.download_as_string())

        # change seconds to datetime
        for k,v in list(od.items()):
            newkey = str(datetime.timedelta(seconds=int(k)))
            od[newkey] = od.pop(k)

        client.table(table).update({'detections_by_second': od}).eq('video', f'{file_name}.mp4').execute()

    if 'stats.json' in file:
        stats = json.loads(blob.download_as_string())
        for k,v in stats.items():
            client.table(table).update({k: v}).eq('video', f'{file_name}.mp4').execute()


In [184]:
df = pd.read_csv('patriot_front_data.csv')
print(df)

                              video  Baton  Pepper Spray  Uniformed  \
0                           009.mp4    NaN           NaN        NaN   
1           2021-11-14_01.13.50.mp4    NaN          13.0        NaN   
2         1_4934073479210205678.mp4    NaN           NaN        NaN   
3                           008.mp4    NaN           NaN        6.0   
4           2021-11-20_11.20.14.mp4    NaN           NaN        NaN   
..                              ...    ...           ...        ...   
424                 WA_DSCF3252.mp4    6.0           NaN        5.0   
425             drilling_draft1.mp4    NaN           NaN      130.0   
426         latesT_PAtriotfront.mp4    5.0           1.0      335.0   
427  paint_test_20211208_212840.mp4    NaN           NaN        NaN   
428                     sample2.mp4    NaN           NaN        NaN   

     Chemical Smoke   Gun  Riot Shield  Non Uniformed  spray  running  crowd  \
0               NaN  10.0         13.0          150.0   25.0      N